#Step2. 원본 데이터로만 학습 시 모델 성능 평가

###1.환경 설정 (패키지 설치, 드라이브 마운트)
실험에 필요한 패키지 설치, 드라이브 마운트,
경로/나이대/랜덤시드(T4에서 재현 가능성 확보)설정

In [1]:
!pip install transformers jiwer librosa soundfile accelerate pydub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import json
import glob
import random
import numpy as np
import pandas as pd
import torch

from jiwer import wer, cer

# 1) childrenvoice 루트 경로
ROOT_DIR  = "/content/drive/MyDrive/childrenvoice"
TRAIN_DIR = os.path.join(ROOT_DIR, "traindata")
VAL_DIR   = os.path.join(ROOT_DIR, "valdata")

# 실험 1에서 이미 생성했다고 가정하는 test CSV
TEST_CSV  = os.path.join(ROOT_DIR, "test_age4_12_per_age125_with_text.csv")

# 2) 사용할 나이대 (4~12세)
TARGET_AGES = list(range(4, 13))  # [4,5,6,7,8,9,10,11,12]

# 3) 랜덤 시드 고정 (재현성)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 4) 디바이스 정보 (코랩 T4)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("ROOT_DIR :", ROOT_DIR)
print("TRAIN_DIR:", TRAIN_DIR)
print("VAL_DIR  :", VAL_DIR)
print("TEST_CSV :", TEST_CSV)
print("TARGET_AGES:", TARGET_AGES)
print("Device:", device)

ROOT_DIR : /content/drive/MyDrive/childrenvoice
TRAIN_DIR: /content/drive/MyDrive/childrenvoice/traindata
VAL_DIR  : /content/drive/MyDrive/childrenvoice/valdata
TEST_CSV : /content/drive/MyDrive/childrenvoice/test_age4_12_per_age125_with_text.csv
TARGET_AGES: [4, 5, 6, 7, 8, 9, 10, 11, 12]
Device: cuda


###2. train/val CSV 로드 (무결성 검증·텍스트 추출 완료본 사용)

- 이전 Step에서 만든
  - train_age4_12_all_with_text_cleaned.csv
  - val_age4_12_per_age125_with_text_cleaned.csv
  를 그대로 불러와 사용

In [4]:
import librosa

train_csv_path = "/content/drive/MyDrive/childrenvoice/train_age4_12_all_with_text_cleaned.csv"
val_csv_path   = "/content/drive/MyDrive/childrenvoice/val_age4_12_per_age125_with_text_cleaned.csv"

train_df = pd.read_csv(train_csv_path)
val_df   = pd.read_csv(val_csv_path)

print("train_df shape:", train_df.shape)
print("val_df   shape:", val_df.shape)

train_df shape: (9000, 5)
val_df   shape: (1125, 5)


###3. Dataset 클래스 정의 (원본 waveform 사용)
- ChildSpeechDataset
   - __getitem__에서
     1. wav 로드 (16kHz)
     2. WhisperProcessor로 log-mel 특징 추출
     3. tokenizer로 텍스트 → 토큰
   - trim/pad 사용하지 않음

In [5]:
from torch.utils.data import Dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

class ChildSpeechDataset(Dataset):
    def __init__(self, df: pd.DataFrame, processor: WhisperProcessor, sampling_rate: int = 16000):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.target_sr = sampling_rate

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio_path = row["audio_path"]
        text       = str(row["text"])

        # 1) 오디오 로드 (16kHz, 앞뒤 무음 제거 없음)
        audio, sr = librosa.load(audio_path, sr=self.target_sr)

        # 2) Whisper 입력 특징 (log-mel, 80-bin)
        inputs = self.processor(
            audio,
            sampling_rate=self.target_sr,
            return_tensors="pt",
        )
        input_features = inputs.input_features[0]  # [80, T]

        # 3) 레이블 토큰화
        labels = self.processor.tokenizer(
            text,
            return_tensors="pt",
        ).input_ids[0]

        return {
            "input_features": input_features,
            "labels": labels,
        }

###4. DataCollator + Whisper-small 로드 + Dataset 준비

- Whisper-small + Processor 로드
- 언어/태스크 프롬프트만 설정 (ko, transcribe)
- suppress_tokens / 후처리 사용 X
- DataCollator에서 input_features stack, labels pad & -100 처리ㅁ


In [6]:
from dataclasses import dataclass
from typing import Dict, List

@dataclass
class DataCollatorWhisper:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        # input_features: [B, 80, T] 형태로 stack
        input_features = torch.stack([f["input_features"] for f in features])

        # labels: pad 후 pad_token_id -> -100
        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            padding=True,
            return_tensors="pt",
        )
        labels = labels_batch["input_ids"]
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "input_features": input_features,
            "labels": labels,
        }

# Whisper-small + Processor 로드
MODEL_NAME = "openai/whisper-small"
LANG       = "ko"
TASK       = "transcribe"

processor = WhisperProcessor.from_pretrained(MODEL_NAME, language=LANG, task=TASK)
model     = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)

# 디코더 언어/태스크 프롬프트 고정 (한국어 + transcribe)
forced_ids = processor.get_decoder_prompt_ids(language=LANG, task=TASK)
model.config.forced_decoder_ids = forced_ids
if hasattr(model, "generation_config"):
    model.generation_config.forced_decoder_ids = forced_ids

# suppress_tokens / 후처리 사용 안 함
model.config.suppress_tokens = None
if hasattr(model, "generation_config"):
    model.generation_config.suppress_tokens = None

# (선택) 캐시 끄기
model.config.use_cache = False

model.to(device)

train_dataset = ChildSpeechDataset(train_df, processor, sampling_rate=16000)
val_dataset   = ChildSpeechDataset(val_df,   processor, sampling_rate=16000)

data_collator = DataCollatorWhisper(processor)

print("✅ train 샘플 수:", len(train_dataset))
print("✅ val 샘플 수  :", len(val_dataset))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

✅ train 샘플 수: 9000
✅ val 샘플 수  : 1125


###5. TrainingArguments (원본 waveform · 낮은 lr) + Trainer 정의 & 학습

- lr: 3e-6 (기존 5e-6보다 더 보수적으로)
- num_train_epochs: 5
- early stopping: patience=2
- trim/pad / suppress_tokens / temperature 조작 없음


In [7]:
import numpy as np
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

# ➜ 1차 실험과 구분되는 새로운 출력 디렉터리
output_dir = os.path.join(ROOT_DIR, "models/whisper_small_childrenvoice_notrim_lr3e6")

# 혹시 모를 캐시 관련 이슈 방지
model.config.use_cache = False

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=1,      # effective batch size = 4
    learning_rate=3e-6,                 # 더 낮은 학습률
    warmup_steps=200,
    num_train_epochs=5,                 # early stopping 있으니 5로 충분
    eval_strategy="epoch",              # epoch마다 평가
    save_strategy="epoch",              # epoch마다 체크포인트 저장
    logging_steps=50,
    predict_with_generate=True,
    generation_max_length=225,

    fp16=False,                         # 안정성 우선
    gradient_checkpointing=False,

    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    report_to=["tensorboard"],
    seed=SEED,
    data_seed=SEED,
)

def compute_metrics(eval_pred):
    preds, labels = eval_pred  # 둘 다 numpy array or 리스트

    # 1) 예측 토큰 → 문자열
    pred_str = processor.batch_decode(preds, skip_special_tokens=True)

    # 2) labels: -100 → pad_token_id로 치환 후 디코딩
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    label_str = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

    return {
        "wer": wer(label_str, pred_str)
    }

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics,
    callbacks=[
        EarlyStoppingCallback(
            early_stopping_patience=2,     # 2 epoch 연속 개선 없으면 정지
            early_stopping_threshold=1e-3,
        )
    ],
)

trainer.train()

# 🔹 저장 전에 GenerationConfig 정리 (sampling 파라미터 비우기)
gen_cfg = model.generation_config
if not gen_cfg.do_sample:
    # sampling 전용 파라미터는 greedy 모드에서 None으로 비워두는 게 안전
    if hasattr(gen_cfg, "temperature"):
        gen_cfg.temperature = None
    if hasattr(gen_cfg, "top_k"):
        gen_cfg.top_k = None
    if hasattr(gen_cfg, "top_p"):
        gen_cfg.top_p = None
    if hasattr(gen_cfg, "typical_p"):
        gen_cfg.typical_p = None

# 최종 모델 저장
trainer.save_model(output_dir)
processor.save_pretrained(output_dir)

print("✅ 파인튜닝 완료 및 모델/processor 저장:", output_dir)

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Wer
1,0.279200,0.346509,0.305195
2,0.124700,0.316934,0.275143
3,0.063500,0.310372,0.261703
4,0.032600,0.316926,0.261854
5,0.024700,0.321990,0.262760


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


✅ 파인튜닝 완료 및 모델/processor 저장: /content/drive/MyDrive/childrenvoice/models/whisper_small_childrenvoice_notrim_lr3e6


###6. 파인튜닝된 모델로 test 평가 (전체/나이별 WER, CER)

- train/val과 동일하게 (원본 waveform → WhisperProcessor → generate)
- 후처리 없이 raw 출력 기준으로 WER/CER 계산
- 결과 CSV:
whisper_small_finetuned3_age4_12_per_age125.csv

In [8]:
# 필요 시 재로드 (런타임이 끊겼다면 주석 해제)
model = WhisperForConditionalGeneration.from_pretrained(output_dir).to(device)
processor = WhisperProcessor.from_pretrained(output_dir)

# 언어 프롬프트 재설정 (suppress_tokens는 None 유지)
forced_ids = processor.get_decoder_prompt_ids(language=LANG, task=TASK)
model.config.forced_decoder_ids = forced_ids
if hasattr(model, "generation_config"):
    model.generation_config.forced_decoder_ids = forced_ids

model.config.suppress_tokens = None
if hasattr(model, "generation_config"):
    model.generation_config.suppress_tokens = None

model.to(device)

# 실험 1에서 만든 test 리스트 사용
test_df = pd.read_csv(TEST_CSV)
test_df = test_df[test_df["text"].astype(str).str.strip() != ""].reset_index(drop=True)
print("✅ test 샘플 수:", len(test_df))

pred_texts = []
ref_texts  = []
results    = []

for idx, row in test_df.iterrows():
    audio_path = row["audio_path"]
    ref_text   = str(row["text"]).strip()
    age        = row["age"]
    speaker_id = row["speaker_id"]
    utt_id     = row["utt_id"]

    # 1) 오디오 로드 (테스트도 원본 그대로, trim/pad 없음)
    audio, sr = librosa.load(audio_path, sr=16000)

    # 2) Whisper 입력 특징
    input_features = processor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
    ).input_features.to(device)

    # 3) 추론
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # ✅ 평가용: 후처리 없이 raw 출력 사용
    raw_hyp_text = processor.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0].strip()

    ref_texts.append(ref_text)
    pred_texts.append(raw_hyp_text)

    results.append({
        "age": age,
        "speaker_id": speaker_id,
        "utt_id": utt_id,
        "audio_path": audio_path,
        "ref_text": ref_text,
        "hyp_text": raw_hyp_text,
    })

# 🔹 전체 WER / CER (raw 기준)
test_wer = wer(ref_texts, pred_texts)
test_cer = cer(ref_texts, pred_texts)

print("\n✅ 파인튜닝 Whisper-small 테스트 결과 (age 4~12, 각 나이 최대 125개, notrim & lr=3e-6)")
print(f"전체 WER: {test_wer:.4f}")
print(f"전체 CER: {test_cer:.4f}")

results_df = pd.DataFrame(results)

# 🔹 전체 결과 CSV (2차 실험용 새 파일명)
save_compare_path = os.path.join(
    ROOT_DIR,
    "whisper_small_finetuned3_age4_12_per_age125.csv"
)
results_df.to_csv(save_compare_path, index=False, encoding="utf-8-sig")
print("\n✅ 발화별 비교 결과 CSV 저장 완료:", save_compare_path)

# 🔹 나이별로 최대 20개씩 REF/HYP 출력 (raw 기준)
for age, sub in results_df.groupby("age"):
    sub = sub.sort_values(["speaker_id", "utt_id"]).reset_index(drop=True)

    print(f"\n==================== age {age} (n={len(sub)}) ====================")
    for i, r in sub.head(20).iterrows():
        print(f"[{i}] speaker={r['speaker_id']}, utt_id={r['utt_id']}")
        print(f"REF: {r['ref_text']}")
        print(f"HYP: {r['hyp_text']}")
        print("-" * 50)

# 🔹 나이별 WER/CER (raw 기준)
age_scores = []
for age, sub in results_df.groupby("age"):
    refs = sub["ref_text"].astype(str).tolist()
    hyps = sub["hyp_text"].astype(str).tolist()
    age_wer = wer(refs, hyps)
    age_cer = cer(refs, hyps)
    age_scores.append({
        "age": age,
        "wer": age_wer,
        "cer": age_cer,
        "n_samples": len(sub),
    })

age_scores_df = pd.DataFrame(age_scores).sort_values("age").reset_index(drop=True)
print("\n✅ 나이별 WER/CER:")
print(age_scores_df)

✅ test 샘플 수: 1125


`generation_config` default values have been modified to match model-specific defaults: {'temperature': None, 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.



✅ 파인튜닝 Whisper-small 테스트 결과 (age 4~12, 각 나이 최대 125개, notrim & lr=3e-6)
전체 WER: 0.3373
전체 CER: 0.1896

✅ 발화별 비교 결과 CSV 저장 완료: /content/drive/MyDrive/childrenvoice/whisper_small_finetuned3_age4_12_per_age125.csv

==================== age 4 (n=125) ====================
[0] speaker=1079, utt_id=K00011079-AFG10-L4N1D1-E-K0KK-00107381
REF: 나쁜 먼지가 많아요.
HYP: 나쁜 만지가 맞아요.
--------------------------------------------------
[1] speaker=1079, utt_id=K00021079-AFG10-L4N1D1-E-K0KK-00107404
REF: 큰 소리로 말해요.
HYP: 큰 소리로 말해요.
--------------------------------------------------
[2] speaker=1079, utt_id=K00031079-AFG10-L4N1D1-E-K0KK-00107429
REF: 옷이 얼룩덜룩 더럽혀 있어요.
HYP: 혹시 얼룩 얼룩 더럽혀 있어요?
--------------------------------------------------
[3] speaker=1079, utt_id=K00041079-AFG10-L4N1D1-E-K0KK-00107445
REF: 책을 펴요.
HYP: 차가 파요.
--------------------------------------------------
[4] speaker=1079, utt_id=K00051079-AFG10-L4N1D1-E-K0KK-00107462
REF: 네모난 문을 열어요.
HYP: 할머니는 뭘 알아요?
---------------------------------------